# PandaSDMX

- [Documentazione aggiornata (v1.4.1)](https://pandasdmx.readthedocs.io/en/latest/)
- [Esempio breve (con poche spiegazioni)](https://pandasdmx.readthedocs.io/en/master/example.html)
- [Esempio approfondito (ma non troppo aggiornato)](https://pandasdmx.readthedocs.io/en/latest/walkthrough.html)

## Installazione

- L'ultima versione non funziona con Pydantic 1.8.1 ma richiede 1.7 ([dr-leo/pandaSDMX#204](https://github.com/dr-leo/pandaSDMX/issues/204))

In [ ]:
!pip install pandasdmx pydantic==1.7

## Esempio

In [2]:
import pandas
import pandasdmx

# Per type annotations
import pandasdmx.message
import pandasdmx.model
import pandasdmx.source
import pandasdmx.source.estat

/mnt/tera/ext4/code/sdmx-sandbox/venv/lib/python3.9/site-packages/pandasdmx/remote.py:11: RuntimeWarning: optional dependency requests_cache is not installed; cache options to Session() have no effect
  warn(


È possibile selezionare tra più fonti di dati, tra i quali Eurostat (`ESTAT`).

> __Request__: client di comunicazione tra `pandasdmx` e un server di dati come Eurostat

Come prima cosa, è necessario creare un'istanza di `pandasdmx.Request`:

In [ ]:
eurostat: pandasdmx.Request = pandasdmx.Request("ESTAT")
eurostat

> __Dataflow__: set di metadati relativi a una misura effettuata (ad esempio, `educ_enrl1ad - Students by ISCED level, study intensity and sex`)

> __Message__: risposta HTTPS ricevuta in seguito a una richiesta effettuata ad un server di dati

Poi, scarichiamo _tutti_ i dataflow disponibili usando `.dataflow()` sul client creato in precedenza per effettuare una richiesta al server Eurostat, creando un `pandasdmx.message.Message`:

In [ ]:
all_flows_msg: pandasdmx.message.Message = eurostat.dataflow()
all_flows_msg

> __Series__: una specie di `dict` più veloce e avanzato implementato da `pandas`

PandaSDMX ha la funzionalità che cercavamo di cercare dataset per keyword!

Per effettuare la ricerca, usiamo il metodo `.to_pandas()` per convertire il `Message` in oggetti Python e/o `pandas`, poi usiamo i metodi "nativi" per trovare quello che ci serve:

In [ ]:
# Converte i risultati in due Series di pandas, una con i dataflow e una con la loro relativa struttura
_dict: dict[str, pandas.Series] = all_flows_msg.to_pandas()
all_flows: pandas.Series = _dict["dataflow"]
all_structs: pandas.Series = _dict["structure"]
all_flows, all_structs

In [ ]:
# Cerchiamo nella Series i allflows la cui descrizione contiene "student"
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html
student_flows: pandas.Series = all_flows[all_flows.str.contains("student", case=False)]
student_flows

Per continuare gli esperimenti, prendiamo il primo dataflow tra quelli contenenti `"student"` nel label:

In [ ]:
my_flow_label = student_flows.index[0]
my_flow_label

Usiamo il label per chiamare di nuovo `.dataflow()`, specificando però stavolta il dataflow di cui ci interessano i dettagli:

In [ ]:
my_flow_msg: pandasdmx.message.Message = eurostat.dataflow(my_flow_label)
my_flow: pandasdmx.model.DataflowDefinition = my_flow_msg.dataflow[my_flow_label]
my_flow

> __Structure__: metadati su come sono strutturate le misure di un dataflow (cosa è stato misurato, quali filtri è possibile applicare, note, etc)

_Particolarità di Eurostat: la structure va richiesta separatamente dal dataflow, in quanto tutti i campi a parte `id` di `dataflow.structure` sono sempre vuoti._

Scopriamo prima il label della structure, poi scarichiamo da Eurostat la structure del dataflow che ci interessa con il metodo `.datastructure()`:

In [ ]:
my_struct_label: pandasdmx.source.DataStructureDefinition = my_flow.structure.id
my_struct_msg: pandasdmx.message.Message = eurostat.datastructure(my_struct_label)
my_struct: pandasdmx.source.DataStructureDefinition = my_struct_msg.structure[my_struct_label]
my_struct

Ispezioniamo la structure che abbiamo scaricato, visualizzandola contemporaneamente [sul Data Explorer di Eurostat](https://ec.europa.eu/eurostat/databrowser/view/educ_enrl1ad/default/table?lang=en)

> __Measures__: valori aggregati relativi alle misure effettuate, simili a `COUNT(*)` dell'SQL

> __Dimensions__: filtri applicabili ai dati raccolti in modo simile all'`HAVING` dell'SQL

> __Attributes__: ???

> __Annotations__: commenti che possono essere aggiunti al dataflow

In [ ]:
my_struct.annotations, my_struct.measures, my_struct.attributes, my_struct.dimensions

Infine, richiediamo i dati da Eurostat, limitandoli a quelli dell'`IT`alia dal 2010 in poi e selezionando solo il `WORKTIME` `TOTAL`, e convertiamoli in una Series multi-chiave:

In [ ]:
my_data_msg: pandasdmx.message.Message = eurostat.data(my_flow_label, key={"GEO": "IT", "WORKTIME": "TOTAL"}, params={"startPeriod": "2010"})
my_data_series: pandas.Series = my_data_msg.to_pandas()
my_data_series

> __DataFrame__: Tabella di dati di `pandas`, implementata come array di Series

Per avere una rappresentazione migliore dei dati sul notebook, convertiamo la Series a un DataFrame:

In [ ]:
my_data: pandas.DataFrame = my_data_series.to_frame()
my_data

Inoltre, per semplificarne le query, "appiattiamo" il [MultiIndex](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html) trasformandolo in normalissime colonne:

In [ ]:
my_data.reset_index(inplace=True)
my_data

Abbiamo finalmente i dati, e possiamo manipolarli come un qualsiasi DataFrame di `pandas`, in modo molto simile a una tabella SQL:

In [ ]:
# Il numero di studenti [M]aschi, [F]emmine e [T]otali in Italia nel [2010], [2011] e [2012]
my_data.groupby(["FREQ", "TIME_PERIOD", "SEX"]).first()

## Sorgenti dati

Tra le sorgenti di dati di cui abbiamo parlato, sono [completamente supportate](https://pandasdmx.readthedocs.io/en/latest/sources.html):

- `ESTAT` - Eurostat
- `ISTAT` - ISTAT

Queste sorgenti non supportano lo standard `SDMX-MD` ma solo lo standard `SDMX-JSON`, che [non supporta query di metadati e struttura](https://pandasdmx.readthedocs.io/en/latest/sources.html#data-source-limitations):

- `OECD` - Organisation for Economic Cooperation and Development

## Archiviazione dati

Se si vogliono replicare dati provenienti da queste fonti, si potrebbe usare tranquillamente un database **relazionale** (SQL) le cui tabelle sono generate a runtime in base alla struttura del dataflow desiderato.

[SQLAlchemy](https://www.sqlalchemy.org/) potrebbe essere utile in questo caso; non sono particolarmente familiare con l'[ORM di Django](https://docs.djangoproject.com/en/3.1/topics/db/models/), ma sembrano molto simili (anche se [si direbbe che SQLAlchemy supporti query più complesse](https://stackoverflow.com/questions/18199053/example-of-what-sqlalchemy-can-do-and-django-orm-cannot)).

## Filtraggio in base a `TIME_PERIOD`

È possibile capire se un DataFrame ha una colonna `TIME_PERIOD` in questo modo:

In [ ]:
"TIME_PERIOD" in list(my_data.columns)

I `TIME_PERIOD` possono essere misurati in modi diversi: anni, quadrimestri, giorni, etc...

I valori possibili sono:

In [ ]:
list(my_struct.dimensions.get("FREQ").local_representation.enumerated)

Per capire quali sono disponibili, si può effettuare una query aggregata:

In [ ]:
list(my_data.groupby(["FREQ"]).any().index)

In questo caso, è disponibile solo `A`, il che significa che le misurazioni sono **eseguite solo annualmente**.

Possiamo trovare il "periodo" più recente con una query sulla tabella:

In [ ]:
latest_period = my_data["TIME_PERIOD"].max()
latest_period

Possiamo filtrare i dati in modo da avere solo quelli del periodo desiderato:

In [ ]:
my_data.loc[my_data["TIME_PERIOD"] == latest_period]

In generale, possiamo applicare ulteriori filtri effettuando accessi agli elementi (`__getitem__`) della proprietà `loc` del dataframe:

In [ ]:
my_data.loc[my_data["SEX"] == "M"]

In [26]:
(
    my_data
        .loc[my_data["TIME_PERIOD"] == latest_period]
        .loc[my_data["SEX"] == "M"]
        .loc[my_data["ISCED97"] == "ED0"]
)

,FREQ,UNIT,ISCED97,SEX,WORKTIME,GEO,TIME_PERIOD,value
5,A,NR,ED0,M,TOTAL,IT,2012,879256.0


## Proviamo con l'`ISTAT`

In [5]:
istat: pandasdmx.Request = pandasdmx.Request("ISTAT")
istat_flows_msg: pandasdmx.message.Message = istat.dataflow()
_dict: dict[str, pandas.Series] = istat_flows_msg.to_pandas()
istat_flows: pandas.Series = _dict["dataflow"]
istat_structs: pandas.Series = _dict["structure"]
istat_flows, istat_structs

(101_1015                                                Crops
 101_1030                    PDO, PGI and TSG quality products
 101_1033                                         slaughtering
 101_1039                         Agritourism - municipalities
 101_1077    PDO, PGI and TSG products:  operators - munici...
                                   ...                        
 97_953                   Environmental protection expenditure
 98_1066     Productivity measures - Accounts in the 2014 v...
 98_1067     Productivity measures - Accounts in the 2011 v...
 98_197                                  Productivity measures
 9_951                                    Mining and quarrying
 Length: 458, dtype: object,
 DCSP_COLTIVAZIONI         
 DCSP_DOPIGP               
 DCSP_MACELLAZIONI         
 DCSP_AGRITURISMO_COM      
 DCSP_DOPIGP_COM           
                         ..
 DCCN_SPESAPROTAMB         
 DCCN_PRODUTTIVITA_B14     
 DCCN_PRODUTTIVITA_B11     
 DCCN_PRODUTTIVITA        